In [1]:
#%matplotlib
from classy import Class
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import scipy.integrate as sc
import scipy.optimize as optimize
import scipy.interpolate

import matplotlib.pyplot as plt
import scipy.integrate as inv

import scipy.special as special
from time import time


In [ ]:
tid1 = time()
def TEST(kvec=[1],k_ind=0,z=800,
         mv_test=1 ,Omega_M = 0.99,
         Omega_b=0.01,PLOT_FIGUR=True,
         EdSitter=True,interpolation_type="cubic"):

    if EdSitter==True:
        Omega_M = 0.99
    
    
    def Power(x,k):
        return np.power(x,k)
    def Cos(x):
        return np.cos(x)
    def Sin(x):
        return np.sin(x)
    def CosIntegral(x):
        return special.sici(x)[1]
    def SinIntegral(x):
        return special.sici(x)[0]
    def Sqrt(x):
        return np.sqrt(x)
    def Sinc(x):
        return np.sinc(x)
    
    Omega_lambda = 1 - Omega_M - Omega_b
    cs_factor = (25*special.zeta(5)/special.zeta(3))**0.5
    cs_factor /= 4
    factor = cs_factor
    k = kvec[k_ind]
    
    # CLASS stuff
    H0=67
    Tv_k = 1.947
    # in MeV
    h = 0.6704
    kb = 8.617e-11 # MeV/K
    T_0 = Tv_k * kb
    cosmo_dict ={'H0':H0, 'Omega_cdm':Omega_M,'Omega_b':Omega_b, 'k_output_values':str(kvec).strip('[]')
                 ,"N_ncdm":1,"m_ncdm":mv_test # massen er givet i eV s' jeg ganger min egen test m_v skal være en faktor  1e-6 mindre
               ,'output':'tCl, lCl, dTk, vTk, pCl', 'lensing':'yes',
                "ncdm_fluid_approximation":3
                , "YHe":0.25
                }

    c = 3*1e5 # speed of lign in km/s
    H0 /= c # i 1/Mpc

    cosmo = Class()
    cosmo.set(cosmo_dict)
    cosmo.compute()
    bg = cosmo.get_background()
    tid = bg['proper time [Gyr]'] * c / 306.4
    pts = cosmo.get_perturbations()['scalar']
    # Here we create/import the powerspectrum values from class

    prim = cosmo.get_primordial()
    prim_interp = scipy.interpolate.interp1d(prim["k [1/Mpc]"],prim["P_scalar(k)"], fill_value="extrapolate",kind="linear")

    tranf = cosmo.get_transfer(z=0)
    k = tranf['k (h/Mpc)'] * h
    d_m_af_k =  scipy.interpolate.interp1d(k,tranf["d_m"])
    p = lambda k: 2*np.pi**2 * d_m_af_k(k)** 2 *  prim_interp(k) * h**3 * k**(-3)
    
    
    tranf05 = cosmo.get_transfer(z=1) # så a = 1/ (1+1) 0 0.5
    k05 = tranf['k (h/Mpc)'] * h
    d_m_af_k_05 =  scipy.interpolate.interp1d(k,tranf05["d_m"])
    p05 = lambda k: 2*np.pi**2 * d_m_af_k05(k)** 2 *  prim_interp(k) * h**3 * k ** (-3)
    
    tranf01 = cosmo.get_transfer(z=1) # så a = 1/ (1+1) 0 0.5
    k01 = tranf['k (h/Mpc)'] * h
    d_m_af_k01 =  scipy.interpolate.interp1d(k,tranf01["d_m"])
    p01 = lambda k: 2*np.pi**2 * d_m_af_k01(k)** 2 *  prim_interp(k) * h**3 * k ** (-3)
    
    mv = mv_test * 1e-6

    def cs(a):
        Tv_k = 1.947 # kelvin # MeV
        kb = 8.617e-11 # MeV/K
        T_0 = Tv_k * kb
        mv = mv_test * 1e-6
        cs_factor = (25*special.zeta(5)/special.zeta(3))**0.5
        cs_factor /= 4
        return cs_factor *T_0 / a / mv
    
    def H(a):
        return H0 * np.sqrt(Omega_M /a + Omega_lambda * a**2)    
    
     
    def K(k1,t,theta):
        k2 = k1 * t
        bn = 1
        an = 2/7
        res= (bn - an) + bn/2 *  np.cos(np.pi-theta) * (k2/k1 + k1/k2) + an * (np.cos(np.pi-theta))**2
        return res 
    
        
    def Bispektrum(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  (2*p(k1)*p(k2)*K(k1,t,theta) +2*p(k3)*p(k1)*K(k3,t3,theta3) +2*p(k2)*p(k3)*K(k2,t2,theta2))
        return res
    
    def Bispektrum05(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  (2*p05(k1)*p05(k2)*K(k1,t,theta) +2*p05(k3)*p05(k1)*K(k3,t3,theta3) +2*p05(k2)*p05(k3)*K(k2,t2,theta2))
        return res
    def Bispektrum01(k1,t,theta):
        k2 = t * k1
        phi = np.pi - theta
        k3 = np.sqrt((k2*np.sin(phi))**2 + (k2*np.cos(phi)+k1)**2)    
        theta2 = np.arccos((k3**2 + k2**2 - k1**2) / ( 2 * k3 * k2 )) 
        theta3 = np.pi - theta2 - theta
        t2 = k3/k2
        t3 = k1/k3
        res =  (2*p01(k1)*p01(k2)*K(k1,t,theta) +2*p01(k3)*p01(k1)*K(k3,t3,theta3) +2*p01(k2)*p01(k3)*K(k2,t2,theta2))
        return res
    
    plt.figure(13241324123123)
    theta = np.linspace(0,np.pi/2,100)
    k1 = np.logspace(-2,1,20) / h
    plt.title("Equilateral Matter Bispectra, t= 1,")
    k6 = np.array([i * i*i*i*i*i for i in k1])
    plt.loglog(k1,k6 * Bispektrum(k1,1,np.pi/3),label="a = 1")
    plt.loglog(k1,k6 * Bispektrum05(k1,1,np.pi/3),label="a = 0.5")
    plt.loglog(k1,k6 * Bispektrum01(k1,1,np.pi/3),label="a = 0.1")
    plt.ylim([1e-17,1e7])
    plt.xlabel("$k_1$ [h/Mpc]")
    plt.ylabel("$k^6 B(k_1,t,\\theta)$")
    plt.legend()
    

kvec1 = list(np.logspace(-4,2,30))
k =np.logspace(np.log10(kvec1[0]),np.log10(kvec1[-1]),100)
#print(k[0],k[-1])
#print(kvec1[0],kvec1[-1])
TEST(kvec = kvec1,mv_test=0.03)
tid2 = time()

print(np.round(tid1-tid2),2)